In [13]:
import os

import pandas as pd
from langdetect import detect

import src.mongos as mg


%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Const

In [24]:
db_name = 'youtube_comment'

vids = [  # vid(video_ids) goes to collection name
    'XsX3ATc3FbA',  # 작은 것들을 위한 시
    'rBG5L7UsUxA',  # 데뷔곡: 2 COOL 4 SKOOL
]

cids = [
    'comments',
#     'comments_XsX3ATc3FbA',
    'comments_rBG5L7UsUxA',
]

# Load raw comments

In [15]:
def get_df_from_vid(vid):
    return mg.read_mongo(db_name, vid)  # comments, nextPageToken, savedDateTime

# Transform comments

In [16]:
def add_token_and_date_to_comments(row):
    comments = row['comments']  # list
    token = row['nextPageToken']  # string
    dt = row['savedDateTime']  # dt   
    
    result_from_replies = []
    for c in comments:
        replies = c.get('replies', [])
        replies2 = [{**r, 'nextPageToken': token, 'savedDateTime': dt} for r in replies]
        result_from_replies.extend(replies2)
    
    result_from_comments = [{**cmt, 'nextPageToken': token, 'savedDateTime': dt} for cmt in comments]
    result = result_from_replies + result_from_comments
    return result


def get_comments_from_series_list_cmt(series_list_cmt):
    data = [cmt for l in series_list_cmt for cmt in l]; data
    return pd.DataFrame(data)
    

# series_list_cmt = sample.apply(lambda x: add_token_and_date_to_comments(x), axis=1);series_list_cmt
# raw_comments = get_comments_from_series_list_cmt(series_list_cmt); raw_comments

# replies = raw_comments['replies'].loc[raw_comments['replies'].notna()]
# comments_from_replies = get_comments_from_series_list_cmt(replies)

# Transform Columns

In [17]:
def rename_columns(comments):
    return comments.rename(columns={'639-1': 'langCode'})


def drop_columns(comments):
    return comments.drop('replies', axis=1)

# Calculate date time

In [18]:
from datetime import timedelta
import re


def add_date_time(row):
    saved = row['savedDateTime']
    time = row['time']
    obj = re.search(r'(\d+)\s', time)
    if obj:
        value = int(obj.group(1))
    value_in_seconds = value
#     factor_table = {'second': 1, 'minute': 60, 'day': 60*24, 'month': 60*24*30, 'year': 60*24*365}
    # 고치기: second, minute 빼고 전부 60 곱하기
    factor_table = {'second': 1, 'minute': 60, 'hour': 60*60, 'day': 60*60*24, 'month': 60*60*24*30, 'year': 60*60*24*365}
    for k, v in factor_table.items():
        if k in time:
            value_in_seconds = value * v
            break
    return saved - timedelta(seconds=value_in_seconds) + timedelta(hours=9)  # Korean Time

In [19]:
# comments[['savedDateTime', 'time']].apply(add_date_time, axis=1)
# re.search(r'(\d+)\s', '4 minutes ago').group(1)

# Load distinct tokens

In [20]:
def get_token_set(db_name, collection_name,):
    db = mg._connect_mongo(db_name)
    collection = db[collection_name]

    pipeline = [
        {"$group": {"_id": "$nextPageToken"}}
    ]

    cursor = collection.aggregate(pipeline)
    existing = list(cursor)
    existing = set(x['_id'] for x in existing)

    return existing


# existing = get_token_set(db_name, 'comments')
# merged = get_token_set(db_name, vids[0])

# new = list(merged - existing)
# len(new)

# Execution

In [21]:
# 0: 신곡
# 1: 데뷔곡
index = 0

In [22]:
existing = get_token_set(db_name, cids[index])
merged = get_token_set(db_name, vids[index])

new = list(merged - existing)
len(new)

99100

In [23]:
# raw = get_df_from_vid(vids[0]); raw.head()
raw = mg.read_mongo(db_name, vids[index], query={'nextPageToken': {'$in': new}},)

# db = mg._connect_mongo(db_name)
# collection = db[vids[0]]
# cursor = collection.find({'nextPageToken': {'$in': list(new)}}).limit(10)

DocumentTooLarge: BSON document too large (30775429 bytes) - the connected server supports BSON document sizes up to 16777216 bytes.

In [12]:
raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4542 entries, 0 to 4541
Data columns (total 3 columns):
comments         4542 non-null object
nextPageToken    4541 non-null object
savedDateTime    4542 non-null datetime64[ns]
dtypes: datetime64[ns](1), object(2)
memory usage: 106.5+ KB


In [13]:
sample = raw.head()
sample

,comments,nextPageToken,savedDateTime
0,"[{'id': 'UgxftD7mowdpJLbM2Bd4AaABAg', 'author'...",EiYSC3JCRzVMN1VzVXhBwAEAyAEA4AEBogINKP________...,2019-04-19 01:20:52.266
1,"[{'id': 'Ugz3LAvphA3rm_MdlwV4AaABAg', 'author'...",EiYSC3JCRzVMN1VzVXhBwAEAyAEA4AEBogINKP________...,2019-04-19 01:20:53.717
2,"[{'id': 'UgxMTRrfF9obzTXsviZ4AaABAg', 'author'...",EiYSC3JCRzVMN1VzVXhBwAEAyAEA4AEBogINKP________...,2019-04-19 01:20:54.589
3,"[{'id': 'Ugz7ItDRx2sEiYzDF9t4AaABAg', 'author'...",EiYSC3JCRzVMN1VzVXhBwAEAyAEA4AEBogINKP________...,2019-04-19 01:20:56.012
4,"[{'id': 'Ugw80Hq1Pygc62R8HK14AaABAg', 'author'...",EiYSC3JCRzVMN1VzVXhBwAEAyAEA4AEBogINKP________...,2019-04-19 01:20:56.886


In [14]:
series_list_cmt = raw.apply(lambda x: add_token_and_date_to_comments(x), axis=1);series_list_cmt
comments = get_comments_from_series_list_cmt(series_list_cmt); comments.head()

,author,authorLink,authorThumb,edited,hasReplies,id,likes,nextPageToken,numReplies,replies,repliesToken,savedDateTime,text,time,timestamp
0,mundo life,/channel/UCr7LN3wo5LdqObXzUv8gDZg,https://yt3.ggpht.com/-lSf1Oj7hGWQ/AAAAAAAAAAI...,False,False,UgxftD7mowdpJLbM2Bd4AaABAg,0,EiYSC3JCRzVMN1VzVXhBwAEAyAEA4AEBogINKP________...,NaN,NaN,NaN,2019-04-19 01:20:52.266,Aaaaa,10 minutes ago,1.555636e+12
1,Miny E,/channel/UCvG3ZzpNU2bQQtNZV6k3CVw,https://yt3.ggpht.com/-dx6GY_PDpV8/AAAAAAAAAAI...,False,False,Ugyg7hl7kQEe6s19rXt4AaABAg,0,EiYSC3JCRzVMN1VzVXhBwAEAyAEA4AEBogINKP________...,NaN,NaN,NaN,2019-04-19 01:20:52.266,Rumo aos 100mi <3,25 minutes ago,1.555635e+12
2,betsabe Matines,/channel/UCj3ZAAM9_w4wSROloc7Y0ew,https://yt3.ggpht.com/-jX43RPvo0zo/AAAAAAAAAAI...,False,False,UgxUUTXMC7b1yjzT_714AaABAg,0,EiYSC3JCRzVMN1VzVXhBwAEAyAEA4AEBogINKP________...,NaN,NaN,NaN,2019-04-19 01:20:52.266,Que lindo ráp moster lo amo like si el es tu ...,32 minutes ago,1.555635e+12
3,Laura Rodriguez,/channel/UCIRGaz6AgECuDk5P-jEahng,https://yt3.ggpht.com/-hH3yv4h01cA/AAAAAAAAAAI...,False,False,Ugyd4PS85izfANP2qyN4AaABAg,0,EiYSC3JCRzVMN1VzVXhBwAEAyAEA4AEBogINKP________...,NaN,NaN,NaN,2019-04-19 01:20:52.266,¿RM esta ciego?,47 minutes ago,1.555634e+12
4,Kpop Lover99,/channel/UCc6dXjmQAXzEglHBgVv1ndQ,https://yt3.ggpht.com/-esirdXTdaqY/AAAAAAAAAAI...,False,False,UgwcN2-N4ymzhsjGGXF4AaABAg,1,EiYSC3JCRzVMN1VzVXhBwAEAyAEA4AEBogINKP________...,NaN,NaN,NaN,2019-04-19 01:20:52.266,I want go back in this era. Hoho\n\nNo no I ca...,49 minutes ago,1.555634e+12


In [15]:
# comments = rename_columns(comments)
comments = drop_columns(comments)
comments['dateTime'] = comments[['savedDateTime', 'time']].apply(add_date_time, axis=1)

print(len(comments))
# comments

120394


# to_mongo

In [16]:
mg.to_mongo(comments, 'youtube_comment', cids[index])

Inserted rows: 120394


# ...